In [2]:
#! The following code is intented only for illustration and cannot be run as a single script.
import numpy as np
from multiprocessing import Queue
import pandas as pd
import redis
import time
import datetime
import matplotlib.pyplot as plt
import pickle
import os
from os.path import join, isfile
from os import listdir
import seaborn as sns
sns.set()
from itertools import product
import warnings
warnings.filterwarnings('ignore')
import matplotlib.backends.backend_pdf
from tqdm import tqdm
from scipy.stats import zscore, linregress
import pickle
from sklearn.cluster import DBSCAN

In [3]:
def convolve(timeseries, window_small, window_large, bin_size=5, min_occurrences=600):

    series1 = np.convolve(timeseries, np.ones(int(window_small/bin_size)), mode="valid")
    series2 = np.convolve(timeseries, np.ones(int(window_large/bin_size)), mode="valid")
        
    stride = series1.shape[0] - series2.shape[0]
    series1 = series1[stride:]

    result = np.nan_to_num(series1 / series2, nan=0.) * (window_large/window_small)
    result[result == 0] = 1

    # two sided anomalies
    #pos = np.where(result < 1)[0]
    #result[pos] = 1/result[pos]
    
    #print(result)
    
    f = lambda n: 1/(1+np.e**-n)  # normalize to [0;1]
    
    return np.array(list(map(f, result)))
    

In [4]:
def convolve_zscore(timeseries, window_small, window_large, bin_size=5, min_occurrences=600):
    
    length = timeseries.shape[0] - int(window_large/bin_size)
    result = np.zeros(length)

    for i in range(length):
        series = timeseries[i:i + int(window_large/bin_size)]
        result[i] = np.abs(zscore(series)[-1])

    f = lambda n: 1/(1+np.e**-n)  # normalize to [0;1]
    
    return np.array(list(map(f, result)))

In [5]:
def convolve_residual(timeseries, window_small, window_large, bin_size=5, min_occurrences=600):
    
    length = timeseries.shape[0] - int(window_large/bin_size)
    result = np.zeros(length)
    x = np.array([i for i in range(int(window_large/bin_size))])

    for i in range(length):
        series = np.asarray(timeseries[i:i + int(window_large/bin_size)])
        reg = linregress(x, series)
        m = reg.slope
        t = reg.intercept
           
        residual = series[-1] - (m * int(window_large/bin_size) + t)
        residual *= residual
        result[i] = max(residual, 1)
                    
    result = np.log(result)
    f = lambda n: 1/(1+np.e**-n)  # normalize to [0;1]
    
    return np.array(list(map(f, result)))

In [6]:
def thresholds(ratios, memory_size=12*5*24, percentile=90):
    N = int(memory_size / 5)
    percentiles = []
    
    for i in range(len(ratios) - N):
        percentiles.append(np.percentile(ratios[i:i+N], percentile))
        
    percentiles = np.asarray(percentiles)
    percentiles[percentiles == np.inf] = 0
    
    return percentiles

In [31]:
def temporal_clustering(percentile=95, minSamples=3, eps=600):
    path = "/home/../realTweets/"

    df = pd.DataFrame(columns=["timestamp", "value", "anomaly_score", "token"])

    files = [join("tmp/", f) for f in listdir("tmp/") if isfile(join("tmp/", f))]

    for file in files:
        tmp = pd.read_csv(file, delimiter=",")
        tmp = tmp[["timestamp", "value", "anomaly_score"]]
        tmp["token"] = file.split("volume_")[-1].split(".")[0]
        df = pd.concat((df, tmp))
        
    for token in df["token"].unique():

        # filter on timestamps where anomaly_score > threshold
        df_ = df[df["token"] == token]
        threshold = np.nanpercentile(df_["anomaly_score"], percentile)
        relevant_idx = np.where(df_["anomaly_score"] > threshold)[0]
        df_ = df_.iloc[relevant_idx]
        
        

        # convert string date to timestamp
        f = lambda n: int(time.mktime(datetime.datetime.strptime(n, "%Y-%m-%d %H:%M:%S").timetuple()))
        timestamps = df_["timestamp"].apply(lambda n: f(n))

        # temporal clustering
        ts = np.asarray(sorted(timestamps)).reshape(-1, 1)
        # dbscan = DBSCAN(min_samples=minSamples, eps=max(eps, minSamples*300)).fit(ts)
        dbscan = DBSCAN(min_samples=minSamples, eps=minSamples*eps).fit(ts)
        labels = dbscan.labels_

        # adjust anomaly scores for clustered anomalies:
        idx = np.where(labels > -1)

        # TODO: extrahierte Daten müssen noch in die relevanten Zeiträume zurückgeschrieben werden
        df_ = df[df["token"] == token]
        relevant_idx = relevant_idx[idx[0]]
        df_.loc[relevant_idx, "anomaly_score"] = 1  #(2/3)
        #df_.loc[~relevant_idx, "anomaly_score"] **= (1/2)

        # re-persist data
        path = "/home/../realTweets/adbc_Twitter_volume_" + token + ".csv"
        df_ = df_.drop(['token'], axis=1)
        df_.to_csv(path)


In [29]:
def process_dataset(path_to_dataset, path_to_output_file, p_small, p_large, detector, pdf=None):

    # general parameters
    window_small = p_small
    window_large = p_large

    # load data
    df = pd.read_csv(path_to_dataset)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
    
    scores = None

    
    # convolutions
    if detector == "ratio":
        scores_ratio = convolve(df["value"], window_small, window_large)[1:]
        scores = scores_ratio
    elif detector == "zscore":
        scores_zscore = convolve_zscore(df["value"], window_small, window_large)
        scores = scores_zscore
    elif detector == "residual":
        scores_residual = convolve_residual(df["value"], window_small, window_large)    
        scores = scores_residual
    elif detector == "combined":
        scores_ratio = convolve(df["value"], window_small, window_large)[1:]
        scores_zscore = convolve_zscore(df["value"], window_small, window_large)
        scores_residual = convolve_residual(df["value"], window_small, window_large)
        scores = (scores_ratio * scores_zscore * scores_residual) ** (1/3)
    elif detector == "two":
        scores_ratio = convolve(df["value"], window_small, window_large)[1:]
        scores_zscore = convolve_zscore(df["value"], window_small, window_large)
        scores = (scores_ratio * scores_zscore) ** (1/2)

    else:
        print("Invalid detector found: {}. Please choose from ['ratio', 'zscore', 'residual']".format(detector))

    stride = df.shape[0] - scores.shape[0]
    scores = np.insert(scores, 0, [0] * stride, axis=0)
    df["anomaly_score"] = scores
    
    # write results to csv
    df.to_csv(path_to_output_file, sep=",", index=False)
    df.to_csv("tmp/" + path_to_output_file.split("/")[-1], sep=",", index=False)
    
    # optional plotting
    if pdf is not None: 
        # plot original timeseries
        sns.set(rc={'figure.figsize':(15.7,8.27)})
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        sns.lineplot(x="timestamp", y="value", data=df, ax=ax1)

        percentile=95

        # plot confidence bands
        t_upper = thresholds(scores, percentile=percentile)
        t_lower = thresholds(scores, percentile=100-percentile)

        stride = df.shape[0] - t_upper.shape[0]

        #ax2.tick_params(axis="y", labelcolor="red")
        ax2.plot(df["timestamp"][stride:], t_lower, color="black", alpha=0.3)
        ax2.plot(df["timestamp"][stride:], t_upper, color="black", alpha=0.3)
        ax2.fill_between(df["timestamp"][stride:], t_lower, t_upper, alpha=0.2, color="black")

        # plot convolutions
        sns.lineplot(x="timestamp", y="anomaly_score", data=df, color="red",  linewidth=1, ax=ax2, alpha=0.4)
        ax2.set_ylabel("Ratio")
        ax2.tick_params(axis="y", labelcolor="red")
        # ax2.set_ylim([-.3, window_large/window_small * 1.2])
        #ax2.set_ylim([-3.5, 3.5])



        # plot anomaly circles
        x_anom = np.squeeze(np.argwhere(scores[stride:] > t_upper))
        y_anom = scores[stride:][x_anom]

        ax2.scatter(df["timestamp"][stride:].iloc[x_anom], y_anom, color="k")


        plt.title("Visual Results for '{}' (Conv: {}/{}) {}@{}".format("/".join(path_to_dataset.split("/")[-2:]), p_small, p_large, y_anom.shape[0], percentile))
        #plt.savefig(path_to_output_file.replace("csv", "pdf").replace("results", "plots"), bbox_inches='tight')
        pdf.savefig(fig, bbox_inches='tight')

In [ ]:
path_to_nab = "/home/../NAB/"
algo_name = "adbc"
path_to_own_algo = join(path_to_nab + "results/", algo_name)
verbosity = 0

pdf = matplotlib.backends.backend_pdf.PdfPages("anomaly_plots.pdf")

# create folders for plots
try:
    os.mkdir(join(path_to_nab, "plots"))
except:
    ...

try:
    os.mkdir(join(path_to_nab + "plots", algo_name))
except:
    ...

# create folders for algorithm if not present yet
existing_algorithms = [f for f in os.listdir(path_to_nab + "results/") if not isfile(join(path_to_nab + "results/", f))]

if algo_name not in existing_algorithms:
    os.mkdir(path_to_own_algo)


# iterate over all dataset folders
dataset_folders = [f for f in os.listdir(path_to_nab + "data/") if not isfile(join(path_to_nab + "data/", f))]


results = {}
# grid search
vals_small = [5,10,30]
#vals_large = list(np.arange(60, 60*24*4, 60))
vals_large = [720,1440,2160,2880,3600,4320,5040,5760]
detectors = ["zscore", "ratio", "residual", "combined", "two"]


for detector in detectors:

    for s, l in tqdm(product(vals_small, vals_large)):
            
        if s > 5 and detector != "ratio":
            continue

        if verbosity >= 1:
            print("Starting GridSearch - Params: {}/{} with detector {}".format(s, l, detector))

        for folder in dataset_folders[-1:]:

            # create folders for algorithm results if not present yet
            try:
                os.mkdir(join(path_to_own_algo, folder))
            except:
                ...
            try:
                os.mkdir(join(join(path_to_nab + "plots", algo_name), folder))
            except:
                ...

            if verbosity >= 2:
                print("Currently processing Datasets from Folder '{}'.".format(folder))

            # process individual datasets
            datasets = [f for f in os.listdir(join(path_to_nab + "data/", folder)) if isfile(join(join(path_to_nab + "data/", folder), f))]

            for dataset in datasets[:]:
                if verbosity >= 2:
                    print("Next Dataset: '{}'".format(dataset))
                path_to_dataset = join(join(path_to_nab + "data/", folder), dataset)
                path_to_output_file = join(join(path_to_own_algo, folder), algo_name + "_" + dataset)
                #process_dataset(path_to_dataset, path_to_output_file, s, l, pdf)  
                process_dataset(path_to_dataset, path_to_output_file, s, l, detector)  
                
        # benchmark results
        command = "cd NAB/; python3 run.py -d adbc --optimize --score --normalize --skipConfirmation"
        ret = os.popen(command).read()

        acc = -1
        fp = -1
        fn = -1

        for r in ret.split("\n"):
            if "standard' profile = " in r:
                acc = float(r.split("standard' profile = ")[-1])
            elif "reward_low_FP_rate' profile = " in r:
                fp = float(r.split("reward_low_FP_rate' profile = ")[-1])
            elif "reward_low_FN_rate' profile = " in r:
                fn = float(r.split("reward_low_FN_rate' profile = ")[-1])

        results[len(results)] = {"detector": detector,
                                 "params": {"window_small": s, "window_large": l}, 
                                 "results": {"acc": acc, "fp": fp, "fn": fn},
                                "clustering": False}


        if verbosity >= 0:
            print("Baseline Results for GridSearch - Params: {}/{} with detector {} - Acc: {}, FP: {}, FN: {}".format(s, l, detector, acc, fp, fn))
                
        # apply clustering
        for perc in [98, 98.5, 99, 99.5, 99.6, 99.7, 99.8, 99.9]:
            for minSamples in list(np.arange(8, 20, 1)):
                for eps in [300, 450, 600, 750]:

                    temporal_clustering(percentile=perc, minSamples=minSamples, eps=eps)

                    # benchmark results
                    command = "cd NAB/; python3 run.py -d adbc --optimize --score --normalize --skipConfirmation"
                    ret = os.popen(command).read()

                    acc = -1
                    fp = -1
                    fn = -1

                    for r in ret.split("\n"):
                        if "standard' profile = " in r:
                            acc = float(r.split("standard' profile = ")[-1])
                        elif "reward_low_FP_rate' profile = " in r:
                            fp = float(r.split("reward_low_FP_rate' profile = ")[-1])
                        elif "reward_low_FN_rate' profile = " in r:
                            fn = float(r.split("reward_low_FN_rate' profile = ")[-1])

                    results[len(results)] = {"detector": detector,
                                             "params": {"window_small": s, "window_large": l}, 
                                             "results": {"acc": acc, "fp": fp, "fn": fn},
                                            "clustering": True,
                                            "percentile": perc,
                                            "minSamples": minSamples,
                                            "eps": eps}


                    if verbosity >= 0:
                        print(perc, minSamples, eps)
                        print("Clustering Results for GridSearch - Params: {}/{} with detector {} - Acc: {}, FP: {}, FN: {}".format(s, l, detector, acc, fp, fn))


pickle.dump(results, open("results_with_clustering.pcl","wb"))  # all three detectors
# pickle.dump(results, open("results_two_detectors_one_sided.pcl","wb"))  # best two detectors
# pickle.dump(results, open("results_detector.pcl","wb"))  # individual detectors

pdf.close()

0it [00:00, ?it/s]

Baseline Results for GridSearch - Params: 5/720 with detector zscore - Acc: 50.31, FP: 47.3, FN: 64.25
98 8 300
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 48.31, FP: 41.77, FN: 62.95
98 8 450
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 46.98, FP: 39.11, FN: 62.07
98 8 600
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 44.54, FP: 35.78, FN: 60.72
98 8 750
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 43.79, FP: 35.78, FN: 60.23
98 9 300
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 49.35, FP: 43.73, FN: 63.65
98 9 450
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 48.35, FP: 41.73, FN: 62.98
98 9 600
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 45.4, FP: 35.83, FN: 61.02
98 9 750
Clustering Results for GridSearch - Params: 5/720 with detector zscore - Acc: 44.42,

In [25]:
results

{0: {'detector': 'zscore',
  'params': {'window_small': 5, 'window_large': 720},
  'results': {'acc': 50.31, 'fp': 47.3, 'fn': 64.25},
  'clustering': False},
 1: {'detector': 'zscore',
  'params': {'window_small': 5, 'window_large': 720},
  'results': {'acc': 50.31, 'fp': 47.3, 'fn': 64.25},
  'clustering': True,
  'percentile': 99.5,
  'minSamples': 8,
  'eps': 300},
 2: {'detector': 'zscore',
  'params': {'window_small': 5, 'window_large': 720},
  'results': {'acc': 50.31, 'fp': 47.3, 'fn': 64.25},
  'clustering': True,
  'percentile': 99.5,
  'minSamples': 8,
  'eps': 450},
 3: {'detector': 'zscore',
  'params': {'window_small': 5, 'window_large': 720},
  'results': {'acc': 50.31, 'fp': 47.3, 'fn': 64.25},
  'clustering': True,
  'percentile': 99.5,
  'minSamples': 8,
  'eps': 600},
 4: {'detector': 'zscore',
  'params': {'window_small': 5, 'window_large': 720},
  'results': {'acc': 50.31, 'fp': 47.3, 'fn': 64.25},
  'clustering': True,
  'percentile': 99.5,
  'minSamples': 8,
  'e